# The Battle of Neighborhoods (Week 1)

# 1. Introduction to the Problem

## Scenario: 

Staying here, living here, dreaming it over here, makes one wonder how close is your next city to your home. Any city in India, there are many things that can be availed like the back of your hand. So, when comparing a city like New York or Toronto one would be very mindful on which city feels like home by not only on the availability of the home made products but also the cost of living matters the most. I would like to explore the option in New York Manhattan as it would be close to place of work and have many eateries too. So, in search of an apartment, the requirements would be a single bedroom with rent not more than 5000 dollars per month. And should have Indian restaurants and grocery shopping with amneties such as gym and community centres and location should be near subways.

## Business Problem: 

Challenges are meeting the price, location and amenties in and around the apartment in New York Manhattan(as mentioned above).

## Related Audience: 

People all around the world who want to move cities, will have similar preferences of their own ethnicity and culture. Here, we could target the Asian
audience even though it's applicable similarly to all who want to join a new company or settle in the city for a long time or look out on a job of thier preference or even business.

# 2. Data related to the Problem

# Data Description:

1. Details of the apartment including price, address, single bedroom & venues
2. Subway stations details & venues
3. Neighbourhood details including the amenties & venues\
 Use Wikipedia to gather the details

# Data usage to solve the problem:

Using Foursquare and geopy data to pinpoint the venues for the above data requirement.
This includes the subway station, apartment price, neighborhood etc.
Converion of location information to latitude and longitude.

Using this, we can solve the questions pertaining to rental pricing options, distance from subway to apartment to place of work.
Also, the neighborhood and other data related to amneties and community centres.

In [3]:
import numpy as np 
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

# All requested packages already installed.



In [5]:
address = 'India Gate, Delhi'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Indian city {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Indian city 28.6128806, 77.2295975.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [6]:
CLIENT_ID = 'F5550W10YWYY3GC5JOT3S31A4FX5YOG2OWKRRUYJZCZG5HCL' 
CLIENT_SECRET = 'V3Y0M533TBT115CPQGYZEWWWM5W22STDFOUBCMWICS4EJ3SO' 
VERSION = '20180605'

In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=F5550W10YWYY3GC5JOT3S31A4FX5YOG2OWKRRUYJZCZG5HCL&client_secret=V3Y0M533TBT115CPQGYZEWWWM5W22STDFOUBCMWICS4EJ3SO&v=20180605&ll=28.6128806,77.2295975&radius=500&limit=100'

In [9]:
results = requests.get(url).json()

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan...,Sculpture Garden,28.612980,77.228247
1,India Gate | इंडिया गेट (India Gate),Monument / Landmark,28.612796,77.229207
2,UPSC Chaat,Snack Place,28.609324,77.227879
3,Veg Gulati Restaurant,Vegetarian / Vegan Restaurant,28.609089,77.230743
4,Children's Park,Playground,28.610664,77.230816
5,National War Memorial,Memorial Site,28.613303,77.232579
6,UPSC Lane,Smoke Shop,28.609346,77.227874
7,Chor Bizzare,Indian Restaurant,28.608837,77.229894
8,Lovely Chuski,Park,28.612823,77.224908
